<a href="https://colab.research.google.com/github/matt-breeden/colab-public/blob/main/interactive_align.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from google.colab import output, files
output.enable_custom_widget_manager()
import matplotlib.pyplot as plt
from ipywidgets import Output, widgets, FloatSlider, Dropdown, Layout

In [ ]:
"""
two types of patients: aligner (a) and brackets (b)

total patients P
aligner patients p_a
brackets patients p_b
P = p_a + p_b

visits per aligner patient per year vpp_a
visits per brackets patient per year vpp_b

total visits V
aligner visits v_a
brackets visits v_b
V = v_a + v_b
v_a = p_a * vpp_a
v_b = p_b * vpp_b

contract amount per aligner patient cpp_a
contract amount per brackets patient cpp_b

total contract revenue C
aligner contract revenue c_a
brackets contract revenue c_b
C = c_a + c_b
c_a = p_a * cpp_a
c_b = p_b * cpp_b

SOC_visits = v_a / V * 100
SOC_patients = p_a / P * 100
"""

def get_SOC_patients_from_SOC_visits(SOC_visits, vpp_a, vpp_b):
  return 100 * (SOC_visits * vpp_b) / (SOC_visits * vpp_b + 100 * vpp_a - SOC_visits * vpp_a)

def get_SOC_visits_from_SOC_patients(SOC_patients, vpp_a, vpp_b):
  return (-100 * SOC_patients * vpp_a) / (SOC_patients * (vpp_b - vpp_a) - 100 * vpp_b)

def calculate_revenue_soc(SOC_visits, V, vpp_a, vpp_b, cpp_a, cpp_b):
    visits_a = SOC_visits * V / 100
    visits_b = (1 - SOC_visits/100) * V
    p_a = visits_a / vpp_a
    p_b = visits_b / vpp_b
    c_a = p_a * cpp_a
    c_b = p_b * cpp_b
    return c_a, c_b

def calculate_patients_soc(SOC_visits, V, vpp_a, vpp_b, cpp_a, cpp_b):
    visits_a = SOC_visits * V / 100
    visits_b = (1 - SOC_visits/100) * V
    p_a = visits_a / vpp_a
    p_b = visits_b / vpp_b
    return p_a, p_b


In [ ]:
test_V = 1000
test_vpp_a = 10
test_vpp_b = 18
test_cpp_a = 5000
test_cpp_b = 5500
test_SOC_patients = 50
test_SOC_visits = get_SOC_visits_from_SOC_patients(test_SOC_patients, test_vpp_a, test_vpp_b)

title = "Share of Chair Effects"
areaLabels = ['Aligner', 'Brackets']
yLabel = 'Share of Chair (SOC)'
xAxisLabels = {'Revenue': 'Total Contract Revenue ($K)', 'Patients': 'Maximum Total Patients'}

def plot_stacked_area_chart(SOC_patients, V, vpp_a, vpp_b, cpp_a, cpp_b, dependent_variable):
    print("\n")

    x = list(range(101))

    SOC_visits = get_SOC_visits_from_SOC_patients(SOC_patients, vpp_a, vpp_b)

    if (dependent_variable == 'Revenue'):
      y_values = np.array([calculate_revenue_soc(get_SOC_visits_from_SOC_patients(SOC, vpp_a, vpp_b), V, vpp_a, vpp_b, cpp_a, cpp_b) for SOC in x]) / 1000
      actual_y_a, actual_y_b = calculate_revenue_soc(SOC_visits, V, vpp_a, vpp_b, cpp_a, cpp_b)
      actual_y = (actual_y_a + actual_y_b) / 1000
    else:
      y_values = np.array([calculate_patients_soc(get_SOC_visits_from_SOC_patients(SOC, vpp_a, vpp_b), V, vpp_a, vpp_b, cpp_a, cpp_b) for SOC in x])
      actual_y_a, actual_y_b = calculate_patients_soc(SOC_visits, V, vpp_a, vpp_b, cpp_a, cpp_b)
      actual_y = actual_y_a + actual_y_b

    y1 = y_values[:, 0]
    y2 = y_values[:, 1]

    plt.stackplot(x, y1, y2, labels=areaLabels, alpha=0.7)
    plt.axhline(y=y1[-1], color='gray', linestyle='--', alpha=0.4)
    plt.axhline(y=y2[0], color='gray', linestyle='--', alpha=0.4)
    plt.scatter([SOC_patients], [actual_y], color='black', marker='o')
    plt.axvline(x=SOC_patients, color='purple', linestyle='--')
    plt.axhline(y=actual_y, color='black', linestyle='--')
    plt.xlabel(yLabel)
    plt.ylabel(xAxisLabels[dependent_variable])
    plt.title(title)
    plt.legend(loc='upper left')
    plt.show()

SOC_patients_slider = FloatSlider(value=test_SOC_patients, min=0, max=100, step=1, description="Share of Chair (patients):", layout=Layout(width='50%', padding='10px'))
V_slider = FloatSlider(value=test_V, min=0, max=10000, step=100, description='Visit Capacity V:', layout=Layout(width='50%', padding='10px'))
vpp_a_slider = FloatSlider(value=test_vpp_a, min=0, max=30, step=1, description='Aligner Visits per Patient:', layout=Layout(width='50%', padding='10px'))
vpp_b_slider = FloatSlider(value=test_vpp_b, min=0, max=30, step=1, description='Brackets Visits per Patient:', layout=Layout(width='50%', padding='10px'))
cpp_a_slider = FloatSlider(value=test_cpp_a, min=0, max=10000, step=100, description='Aligner Contract per Patient:', layout=Layout(width='50%', padding='10px'))
cpp_b_slider = FloatSlider(value=test_cpp_b, min=0, max=10000, step=100, description='Brackets Contract per Patient:', layout=Layout(width='50%', padding='10px'))
dependent_variable_dropdown = Dropdown(options=['Revenue', 'Patients'], value='Revenue', description='Dependent Variable:', layout=Layout(width='30%', padding='10px'))

SOC_patients_slider.style.description_width = '200px'
V_slider.style.description_width = '200px'
vpp_a_slider.style.description_width = '200px'
vpp_b_slider.style.description_width = '200px'
cpp_a_slider.style.description_width = '200px'
cpp_b_slider.style.description_width = '200px'
dependent_variable_dropdown.style.description_width = '200px'

interactive_plot = widgets.interactive(plot_stacked_area_chart,
                                        SOC_patients=SOC_patients_slider,
                                        V=V_slider,
                                        vpp_a=vpp_a_slider,
                                        vpp_b=vpp_b_slider,
                                        cpp_a=cpp_a_slider,
                                        cpp_b=cpp_b_slider,
                                        dependent_variable=dependent_variable_dropdown,
                                        continuous_update=False)

interactive_plot

interactive(children=(FloatSlider(value=50.0, description='Share of Chair (patients):', layout=Layout(padding=…